# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster

try:
    
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set Keyspace
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
session_item_query = """
CREATE TABLE IF NOT EXISTS session_item
(session_id INT, session_item_id INT, artist_name VARCHAR, song_title VARCHAR, song_length DOUBLE,
PRIMARY KEY (session_id, session_item_id))
"""
try:
    session.execute(session_item_query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_item (session_id, session_item_id, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
try:
    results = session.execute("""
    SELECT * FROM session_item
    WHERE session_id=338 AND session_item_id=4
    """)
except Exception as e:
    print(e)
 
for row in results:
    print('| {} | {} | {} | {} | {} | '.format(row.session_id, row.session_item_id, row.artist_name, row.song_title, row.song_length))

| 338 | 4 | Faithless | Music Matters (Mark Knight Dub) | 495.3073 | 


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
user_session_query = """
CREATE TABLE IF NOT EXISTS user_session
(user_id INT, session_id INT, session_item_id INT, artist_name VARCHAR, first_name VARCHAR, last_name VARCHAR, song_title VARCHAR,
PRIMARY KEY (user_id, session_id, session_item_id))
"""
try:
    session.execute(user_session_query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (user_id, session_id, session_item_id, artist_name, first_name, last_name, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
try:
    results = session.execute("""
    SELECT * FROM user_session
    WHERE user_id=10 AND session_id=182
    """)
except Exception as e:
    print(e)
 
for row in results:
    print('| {} | {} | {} | {} | {} | {} | {} |'.format(row.user_id , row.session_id, row.session_item_id, row.artist_name, 
                                                        row.first_name, row.last_name, row.song_title))

| 10 | 182 | 0 | Down To The Bone | Sylvie | Cruz | Keep On Keepin' On |
| 10 | 182 | 1 | Three Drives | Sylvie | Cruz | Greece 2000 |
| 10 | 182 | 2 | Sebastien Tellier | Sylvie | Cruz | Kilometer |
| 10 | 182 | 3 | Lonnie Gordon | Sylvie | Cruz | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
song_user_query = """
CREATE TABLE IF NOT EXISTS song_user
(song_title VARCHAR, first_name VARCHAR, last_name VARCHAR,
PRIMARY KEY (song_title))
"""
try:
    session.execute(song_user_query)
except Exception as e:
    print(e)                  

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user (song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
try:
    results = session.execute("""
    SELECT * FROM song_user
    WHERE song_title='All Hands Against His Own'
    """)
except Exception as e:
    print(e)
 
for row in results:
    print('| {} | {} | {} |'.format(row.song_title, row.first_name, row.last_name))

| All Hands Against His Own | Sara | Johnson |


### Drop the tables before closing out the sessions

In [17]:
# DROP session_item table
try:
    session.execute("""DROP TABLE IF EXISTS session_item""")
except Exception as e:
    print(e)

In [18]:
# DROP user_session table
try:
    session.execute("""DROP TABLE IF EXISTS user_session""")
except Exception as e:
    print(e)

In [19]:
# DROP s_item table
try:
    session.execute("""DROP TABLE IF EXISTS song_user""")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()